In [1]:
graph = sc.textFile('/FileStore/tables/web_Google-02a64.txt')

In [2]:
graph.take(5)

Out[ 3 ]: 
['# Directed graph (each unordered pair of nodes is saved once): web-Google.txt ',
 '# Webgraph from the Google programming contest, 2002',
 '# Nodes: 875713 Edges: 5105039',
 '# FromNodeId\tToNodeId',
 '0\t11342']

In [3]:
# removing the header rows from the dataset
graph = graph.filter(lambda x: '#' not in x)

In [4]:
graph.take(3)

Out[ 6 ]: ['0\t11342', '0\t824020', '0\t867923']

In [5]:
def createGraph(x):
  result = list()
  result.append((x.split('\t')[0], [x.split('\t')[1]]))
  result.append((x.split('\t')[1], []))
  return result

In [6]:
# converting rdd of string into (key, value) pair rdd
graph = graph.flatMap(createGraph)

In [7]:
def reducingNode(x,y):
  return x+y

In [8]:
# doing a reduce by key operation get (key, [connections]) tuple
graph = graph.reduceByKey(reducingNode)

In [9]:
graph.take(3)

Out[ 11 ]: 
[('126766', ['208737', '820208', '893208']),
 ('660804',
 ['36696',
 '212720',
 '259791',
 '266277',
 '285326',
 '345372',
 '367023',
 '429002',
 '461123',
 '475720',
 '507707',
 '527923',
 '530477',
 '625571',
 '681572',
 '718760',
 '726802',
 '747305',
 '790563',
 '905296']),
 ('429477', [])]

In [10]:
# storing the graph in the cache memory since it will be used again and again

In [11]:
graph.cache()

Out[ 13 ]: PythonRDD[10] at RDD at PythonRDD.scala:56

In [12]:
# total  number of node in the graph
graph.count()

Out[ 14 ]: 875713

In [13]:
# converting rdd to form (node, ([connection], initial_rank=1, rank got from other adjacent nodes))

In [14]:
graph = graph.map(lambda x: (x[0], (x[1], 1, 0)))

In [15]:
graph.cache()

Out[ 17 ]: PythonRDD[12] at RDD at PythonRDD.scala:56

In [16]:
def mapper(x):
  node_id = x[0]
  data = x[1]
  connections = data[0]
  rank = data[1] # would be 1 always
  transfer = data[2]
  result = list()
  '''
  rank of all the node that would come
  into this mapper function would be 1
  '''
  if len(connections) > 0:
    transfer = rank/len(connections)
    for connection in connections:
      result.append((connection, (list(), 0, transfer)))
    result.append((node_id, (list(), 0, 0)))
  else:
    result.append((node_id, (list(), 1, 0)))
  return result

In [17]:
ranked_graph = graph.flatMap(mapper)

In [18]:
'''
calling count(action) below because
transformation wouldn't be performed
until an action is called on it
'''

Out[ 20 ]: "\ncalling count(action) below because\ntransformation wouldn't be performed\nuntil an action is called on it\n"

In [19]:
ranked_graph.count()

Out[ 21 ]: 5980752

In [20]:
'''
reducer add up all the transfered 
rank from a given node
'''

In [21]:
def reducer(x,y):
  return (list(), 0, max(x[1], y[1]) + x[2] + y[2])

In [22]:
ranked_graph = ranked_graph.reduceByKey(reducer)

In [23]:
'''
after reducer is called total number of nodes
in graph should be equal to initial number of
nodes i.e 875713
'''

In [24]:
ranked_graph.count()

Out[ 24 ]: 875713

In [25]:
'''
verifying that total rank should be
equal to total number of nodes in the graph
i.e 875713
'''

In [26]:
rank = sc.accumulator(0)

In [27]:
'''
adding ranks of all the node
and adding the accumulator counter
with it to find the cumulative ranks
of the nodes in the graph
'''

In [28]:
def computeRank(x):
  rank.add(x[1][2])
  return True

In [29]:
a = ranked_graph.map(computeRank)

In [30]:
# calling count action so that map transformation is performed and counter is incremented

In [31]:
a.count()

Out[ 28 ]: 875713

In [32]:
# sum of rank of all the node is equal to total number of nodes 

In [33]:
rank.value

Out[ 29 ]: 875712.9999997353

In [34]:
'''
Now we will be using damping factor (x*(0.85) + 0.15)
to find actual ranks 
of all the nodes in the graph
'''

In [35]:
rank_of_nodes = ranked_graph.map(lambda x: (x[0], x[1][2]*(0.85) + 0.15))

In [36]:
rank_of_nodes = rank_of_nodes.sortBy(lambda x: -x[1])

In [37]:
'''
Top 5 nodes with maximum rank
'''

In [38]:
rank_of_nodes.take(5)

Out[ 36 ]: 
[('163075', 1120.4732297287514),
 ('537039', 992.5233354985326),
 ('597621', 956.9958482012617),
 ('605856', 859.1381177279259),
 ('819223', 794.1657440962442)]